In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import argparse
import warnings
import logging
from os.path import join
from collections import defaultdict
sys.path.append('../')
warnings.filterwarnings("ignore")

import numpy as np
from coffea import processor, util
import hist
import mplhep as hep
import seaborn as sns
from matplotlib import pyplot as plt
from cycler import cycler
from hist import Hist
from hist.intervals import ratio_uncertainty
from azh_analysis.utils.plotting import plot_data_vs_mc
from azh_analysis.utils.parameters import get_lumis
import warnings
warnings.filterwarnings('ignore')
#hep.style.use(["CMS", "fira", "firamath"])

plot = True
year = "2018"
lumi = get_lumis()[year]
OS = util.load(f"../output/data_UL_{year}_OS_ub.coffea")
SS = util.load(f"../output/data_UL_{year}_SS_ub_relaxed.coffea")

cat_labels = {
    'tt': r'$ll\tau_h\tau_h$',
    'et': r'$ll e\tau_h$',
    'mt': r'$ll\mu\tau_h$',
    'em': r'$ll e\mu$',
}

In [ ]:
def plot_closure(
    os,
    ss,
    var,
    cat_label,
    var_label,
    logscale=False,
    outfile=None,
    year="1",
    lumi="1",
    blind=False,
):
    hep.style.use(["CMS", "fira", "firamath"])
    colors = {
        "OS Application": "#005F73",
        "SS Validation": "#0A9396",
    }
    
    fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(9, 12),
        dpi=120,
        gridspec_kw={"height_ratios": (4, 1)},
        sharex=True,
    )
    fig.subplots_adjust(hspace=0.07)
    ax.set_prop_cycle(cycler(color=list(colors.values())))
    os, ss = os["reducible", :], ss["data", :]
    os_norm, ss_norm = os.values().sum(), ss.values().sum()
    os.plot1d(ax=ax, histtype="errorbar", xerr=12.5, color="k", marker="s", markersize=5, mfc='#0A9396', mec='#0A9396', capsize=2, label="OS Application", density=True)
    ss.plot1d(ax=ax, histtype="errorbar", xerr=12.5, color='k', marker="o", markersize=5, mfc='#EE9B00', mec='#EE9B00', capsize=2, label="SS Validation", density=True)
    os_vals = os.density()
    ss_vals = ss.density()
    bins = os.axes[0].centers
    y = os_vals / ss_vals
    print(sum(os_vals)/sum(ss_vals))
    yerr = ratio_uncertainty(os_vals, ss_vals, "poisson")

    rax.errorbar(
        x=bins,
        y=y,
        yerr=yerr,
        color="k",
        linestyle="none",
        marker="o",
        elinewidth=1,
    )

    if logscale:
        ax.set_yscale("log")
        ax.set_xscale("log")
    ax.set_ylabel("Counts")
    rax.set_xlabel(var_label)
    ax.set_xlabel("")
    ax.legend()
    rax.set_ylim([0, 2])
    ax.legend(loc="best", prop={"size": 16}, frameon=True)
    ax.get_legend().set_title(f"{cat_label}")
    hep.cms.label("Preliminary", data=True, lumi=lumi, year=year, ax=ax)
    if outfile is not None:
        plt.savefig(outfile, format="pdf", dpi=800)
    plt.show()

var = "m4l_reg"
outdir = f"../plots/data-mc/{year}"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:
        for mass_type in ["cons"]:
            os = sum(OS[var].values())
            os = os[:, "ee" + cat, ::sum, "none", mass_type, :] + os[:, "mm" + cat, ::sum, "none", mass_type, :]
            ss = sum(SS[var].values())
            ss = ss[:, "ee" + cat, ::sum, "none", mass_type, :] + ss[:, "mm" + cat, ::sum, "none", mass_type, :]
            plot_closure(
                os, ss, var, cat_label=cat_label, var_label=r"$m_{ll\tau\tau}^\mathrm{cons}$",
                logscale=False, outfile=None, year=year, lumi=lumi
            )